# Change Detection with Instance Segmentation

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/change_detection.ipynb)

This notebook demonstrates the change detection functionality in GeoAI, which provides instance segmentation and confidence scoring for individual change objects. 

The change detection functionality builds upon the [torchange](https://github.com/Z-Zheng/pytorch-change-models) package developed by Dr. Zhuo Zheng. We have made it much easier to analyze remote sensing imagery and visualize the results.

## Overview

The change detection system provides:

- **Instance Segmentation**: Each change object gets a unique ID
- **Confidence Scores**: Individual confidence values for each detected instance
- **Proper GeoTIFF Output**: Maintains spatial reference information

## Key Features

- Instance-level change detection with unique IDs
- Confidence scoring for quality assessment
- Support for large GeoTIFF files
- Comprehensive analysis capabilities

## Install packages

In [ ]:
# %pip install geoai-py

## Import libraries

In [ ]:
import geoai
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

## Setup

In [ ]:
# Check if CUDA is available
device = geoai.get_device()
print(f"Using device: {device}")

# Set up paths
out_folder = "change_detection_results"
Path(out_folder).mkdir(exist_ok=True)

print(f"Working directory: {out_folder}")

## Download sample data

We'll use NAIP imagery for Las Vegas to demonstrate change detection.

In [ ]:
# Download NAIP imagery
naip_2019_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_naip_2019_a.tif"
naip_2022_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_naip_2022_a.tif"

naip_2019_path = geoai.download_file(naip_2019_url)
naip_2022_path = geoai.download_file(naip_2022_url)

print(f"Downloaded 2019 NAIP: {naip_2019_path}")
print(f"Downloaded 2022 NAIP: {naip_2022_path}")

## Visualize sample data

In [ ]:
# Check raster information
geoai.get_raster_info(naip_2019_path)

In [ ]:
# View the images
geoai.view_raster(naip_2019_path)

In [ ]:
geoai.view_raster(naip_2022_path)

## Initialize Change Detection

Create the change detection system with optimal parameters.

In [ ]:
# Make sure model directory exists
Path("~/.cache/torch/hub/checkpoints/").expanduser().mkdir(parents=True, exist_ok=True)

# Initialize change detection
detector = geoai.ChangeDetection(sam_model_type="vit_h")

# Configure parameters (following the torchange example)
detector.set_hyperparameters(
    change_confidence_threshold=145,
    use_normalized_feature=True,
    bitemporal_match=True,
)

detector.set_mask_generator_params(
    points_per_side=32,
    stability_score_thresh=0.95,
)

print("Change detection system initialized!")

## Run Change Detection

Execute change detection with instance segmentation and confidence scoring.

In [ ]:
# Run change detection
results = detector.detect_changes(
    naip_2019_path,
    naip_2022_path,
    output_path=f"{out_folder}/binary_mask.tif",
    export_probability=True,
    probability_output_path=f"{out_folder}/probability_mask.tif",
    export_instance_masks=True,
    instance_masks_output_path=f"{out_folder}/instance_masks.tif",
    return_detailed_results=True,
    return_results=False,
)

print(f"Change detection completed!")
print(f"Total instances detected: {results['summary']['total_masks']}")
print(f"Image size: {results['summary']['original_shape']}")

## Analyze Results

Display key statistics and quality metrics.

In [ ]:
# Display statistics
if "statistics" in results and results["statistics"]:
    print("Quality Statistics:")
    for metric, stats in results["statistics"].items():
        print(f"  {metric}: mean={stats['mean']:.3f}, std={stats['std']:.3f}")

# Show top instances
if "masks" in results and len(results["masks"]) > 0:
    print("\nTop 5 detected instances:")
    for i, mask in enumerate(results["masks"][:5]):
        print(
            f"  {i+1}. Instance {mask['mask_id']}: "
            f"IoU={mask['iou_pred']:.3f}, "
            f"Stability={mask['stability_score']:.3f}, "
            f"Area={mask['area']} pixels"
        )

## Visualizations

Use the integrated visualization methods for comprehensive analysis.

In [ ]:
# probability visualization
detector.visualize_results(
    naip_2019_path,
    naip_2022_path,
    f"{out_folder}/binary_mask.tif",
    f"{out_folder}/probability_mask.tif",
)

![](https://github.com/user-attachments/assets/e7c00b50-c456-4653-b8ce-0c9ec8f05b7f)

In [ ]:
# Create split comparison visualization
detector.create_split_comparison(
    naip_2019_path,
    naip_2022_path,
    f"{out_folder}/binary_mask.tif",
    f"{out_folder}/probability_mask.tif",
    f"{out_folder}/split_comparison.png",
)

In [ ]:
# Analyze individual instances
instance_stats = detector.analyze_instances(
    f"{out_folder}/instance_masks.tif",
    f"{out_folder}/instance_masks_scores.tif",
    f"{out_folder}/instance_analysis.png",
)

![](https://github.com/user-attachments/assets/ea1f8a51-ea14-415a-9733-78b243061dd3)

## Comprehensive Analysis Report

Generate a detailed analysis report combining all metrics.

In [ ]:
# Create comprehensive analysis report
detector.create_comprehensive_report(results, f"{out_folder}/comprehensive_report.png")

![](https://github.com/user-attachments/assets/629caf85-0713-4e04-8023-f4273edbbb4c)

## One-Click Complete Analysis

For ultimate simplicity, use the complete analysis method.

In [ ]:
# Alternative: Run complete analysis in one step
# This method does everything - detection, file outputs, and all visualizations
complete_results = detector.run_complete_analysis(
    naip_2019_path, naip_2022_path, "complete_analysis_results"
)

## Summary

This notebook demonstrated the change detection functionality in GeoAI with integrated visualization methods:

### Key Features Used:
1. **Change Detection**: Instance segmentation with confidence scoring
2. **Integrated Visualizations**: Built-in methods for comprehensive analysis
3. **Simplified API**: Clean, streamlined interface following geoai patterns
4. **Complete Analysis**: One-click method for full analysis workflow

### Output Files Generated:
- `binary_mask.tif`: Traditional binary change detection
- `probability_mask.tif`: Probability-weighted change detection  
- `instance_masks.tif`: Instance segmentation with unique IDs
- `instance_masks_scores.tif`: Confidence scores for each instance
- `enhanced_probability_results.png`: Comprehensive visualization
- `split_comparison.png`: Before/after split comparison
- `instance_analysis.png`: Individual instance analysis
- `comprehensive_report.png`: Complete analysis report

### Advantages Over Traditional Methods:
1. **Instance-Level Analysis**: Each change object has unique ID and metrics
2. **Quality Assessment**: Confidence scores for filtering and ranking
3. **Rich Visualizations**: Multiple analysis perspectives in one package
4. **Simplified Workflow**: Integrated methods reduce code complexity
5. **Flexible Usage**: From simple detection to comprehensive analysis